## Virginia Crashes and District Court Case Analysis (2017 Data)

### Import packages

In [1]:
import pandas as pd
import re
from simpledbf import Dbf5
from bokeh.io import show, output_notebook
from bokeh.plotting import figure

output_notebook()

PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.


Loading BokehJS ...

In [2]:

from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None


### Read in data

In [3]:
dbf = Dbf5('Virginia_Crashes.dbf')
crash = dbf.to_dataframe()

one = pd.read_csv('district_criminal_2017_anon_00.csv', low_memory= False)
two = pd.read_csv('district_criminal_2017_anon_01.csv', low_memory= False)
three = pd.read_csv('district_criminal_2017_anon_02.csv', low_memory= False)
four = pd.read_csv('district_criminal_2017_anon_03.csv', low_memory= False)
five = pd.read_csv('district_criminal_2017_anon_04.csv', low_memory= False)
six = pd.read_csv('district_criminal_2017_anon_05.csv', low_memory= False)
sev = pd.read_csv('district_criminal_2017_anon_06.csv', low_memory= False)
eig = pd.read_csv('district_criminal_2017_anon_07.csv', low_memory= False)
nin = pd.read_csv('district_criminal_2017_anon_08.csv', low_memory= False)

In [4]:
## Merge nine files into one data frame
district_court_cases_2017 = one.append(two).append(three).append(four).append(five).append(six).append(sev).append(eig).append(nin)

In [5]:
district_court_cases_2017.head()

,HearingDate,HearingResult,HearingPlea,HearingContinuanceCode,HearingType,HearingCourtroom,fips,FiledDate,Locality,Status,DefenseAttorney,Address,Gender,Race,Charge,CodeSection,CaseType,Class,OffenseDate,ArrestDate,Complainant,AmendedCharge,AmendedCode,AmendedCaseType,FinalDisposition,SentenceTime,SentenceSuspendedTime,ProbationType,ProbationTime,ProbationStarts,OperatorLicenseSuspensionTime,RestrictionEffectiveDate,RestrictionEndDate,OperatorLicenseRestrictionCodes,Fine,Costs,FineCostsDue,FineCostsPaid,FineCostsPaidDate,VASAP,FineCostsPastDue,person_id
0,2018-03-21,NaN,NaN,NaN,Preliminary,NaN,99,2017-12-22,COMMONWEALTH OF VA,Bail,NaN,"CHESTERFIELD, VA 23838",Male,Black(Non-Hispanic),DR AFTER DECLARED H.O.- FELONY,B.46.2-357,Felony,O,2017-12-21,2017-12-21,"PEMBERTON, S.E.",NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,366010000000153
1,2018-03-21,NaN,NaN,NaN,Adjudicatory,NaN,99,2017-12-22,COMMONWEALTH OF VA,Released On Summons,NaN,"CHESTERFIELD, VA 23838",Male,Black(Non-Hispanic),FAIL TO OBEY HIGHWAY SIGN,46.2-830,Infraction,NaN,2017-12-21,NaN,"PEMBERTON, S.E.",NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,366010000000153
2,2017-12-27,Finalized,NaN,Other,Arraignment,NaN,99,2017-12-20,COMMONWEALTH OF VA,Released On Recognizance,NaN,"FREDERICKSBURG, VA 22407",Male,Black(Non-Hispanic),CAPIAS FAILED TO APPEAR,18.2-456,Capias,U,2017-12-20,2017-12-21,JUDGE ROBERT E REIBACH,NaN,NaN,NaN,Dismissed,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,343190000000382
3,2018-04-11,NaN,NaN,NaN,Adjudicatory,NaN,99,2017-06-19,COMMONWEALTH OF VA,Bail,PD,"FREDERICKSBURG, VA 22407",Male,Black(Non-Hispanic),"DWI, 1ST",A.18.2-266,Misdemeanor,1,2017-06-16,2017-06-17,"NOWACKI, A.",NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,343190000000382
4,2018-04-11,NaN,NaN,NaN,Adjudicatory,NaN,99,2017-06-19,COMMONWEALTH OF VA,Released On Summons,NaN,"FREDERICKSBURG, VA 22407",Male,Black(Non-Hispanic),CIVIL VIOLATION - REFUSED BLOOD/BREATH TEST,18.2-268.3,Civil Violation,NaN,2017-06-16,NaN,"NOWACKI, A.",NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,343190000000382


In [6]:
len(district_court_cases_2017)

2100530

#### For this analysis, I'm interested in speeding charges.  Speeding is represented in many ways in the 'Charge' column.  My first step is identify only the columns with speeding, then breakout the alleged speed vs posted speed into new columns

In [7]:
district_court_cases_2017['Charge'].sort_values().unique().tolist()

['"REJECTION" INSPECTION STICKER',
 '"SHORTY" RUNNING AT LARGE',
 '"VIAJERO" RUNNING AT LARGE',
 '$500 REPLACEMENT COST FOR DEER',
 "'AFETY BELT VIOLATION",
 "'ANUF/ETC CONTROLLED SUBSTANCE",
 "'URCHASE/POSSESS ALCOHOL",
 '(1) DOG NO COUNTY LICENSE',
 '(1) DOG RABIES VACCINATIONS',
 '(1) DOG RUNNING AT LARGE',
 '(1) UNDERSIZE BLACK DRUM',
 '(1) UNDERSIZE BLACK SEA BASS',
 '(1) UNDERSIZE GREY TROUT',
 '(1) UNDERSIZE SPECKLE TROUT',
 '(100/55) RECKLESS',
 '(103/55) RECKLESS',
 '(45/25) RECKLESS',
 '(48/25) RECKLESS',
 '(65/45) RECKLESS',
 '(66/45) RECKLESS',
 '(67/45) RECKLESS',
 '(68/45 SPEEDING',
 '(68/45) RECKLESS',
 '(68/45) SPEEDING',
 '(69/45) RECKLESS',
 '(69/450 RECKLESS',
 '(70/45) RECKELSS',
 '(70/45) RECKLESS',
 '(70/50) RECKLESS',
 '(71/45) RECKLESS',
 '(71/50) RECKLESS',
 '(72/45) RECKLESS',
 '(72/45) SPEEDING',
 '(73/45) RECKLESS',
 '(73/50) RECKLESS',
 '(74/45) RECKLESS',
 '(74/50) RECKLESS',
 '(75/45) RECKLESS',
 '(75/50) RECKLESS',
 '(75/55) RECKLESS',
 '(76/45) RECKLESS

- OPR VEH WHILE TXT/READ-1ST 

- 'RD-SP IN EXCESS 80 MPH-MISD'
- ##/## RD
- ##/## SP
- 'RECKLESS DRIVING 78/55 (HSC)'
- '##/## SPEEDING HSC'
- '##/## SPEEDING (HSC)'
- ##/## SPEEDING
- ##/## RECKLESS DRIVING
- RECKLESS DRIVING ##/##
- ##/## RECKLESS SPEED (HSC)
- ##/## SPEEDING HSC
- RD-SP 20/MORE ABOVE SP.LMT-MIS
- ##-## SP
- RECKLESS/SPEEDING ##/##
- ##/## RECKLESS DRIVING SPEED
- ##/## R/D
- ##/## RECKLESS
- ##/## RECKLESS SPEEDING
- RECK DR; ENDANG LIFE/LIMB/PROP
- RECKLESS DRIVING (HSC)
- ###/## RD
- RECKLESS - GENERAL
- RECK RD: RACING
- RD-FAIL TO MAINTAIN CONTROL
- (##/##) RECKLESS
- (##/##) SPEEDING
- ###/## SP BUS/RES DIS ##-## MPH
- ###/## RSD
- ###/## RD-SPEEDING
- ###/## RECKLESS BY SPEED
- ## ## SP
- ## ## RD
- ## ## 
- R/D ##/##
- R.D. ##/##
- R.D ##/##
- R D by SP ##/##

In [8]:
district_court_cases_2017['Speed'] = district_court_cases_2017['Charge'].str.extract(r'([0-1]?\d\d/\d*)')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [9]:
district_court_speeding_cases_2017 = district_court_cases_2017[~district_court_cases_2017['Speed'].isnull()].reset_index()

## Some charges do not note speed and limit.  Dropping those charges

district_court_speeding_cases_2017 = district_court_speeding_cases_2017[district_court_speeding_cases_2017['Speed'] != '20/']

In [10]:
df = district_court_speeding_cases_2017.groupby('person_id')['OffenseDate'].nunique().reset_index(drop = False)
print("There were " + str(len(district_court_speeding_cases_2017)) + " speeding cases across Virginia in 2017")
print("Those speeding cases where commited by " + str(len(df)) + " people")
print("Which means " + str(len(district_court_speeding_cases_2017) - len(df)) + " Repeat Offenders didn't learn their lesson")

There were 618472 speeding cases across Virginia in 2017
Those speeding cases where commited by 575616 people
Which means 42856 Repeat Offenders didn't learn their lesson


In [11]:
#Investigate Crash data and try to marry to speeding data
crash.head()

,OBJECTID,CRASH_DOCU,CRASH_DATE,COLLISION_,CRASH_SEVE,CRASH_WEAT,VDOT_PROPE,WORK_ZONE,NO_OF_VEHI,LIGHT_COND,SURFACE_CO,RELATION_T,SCHOOL_ZON,NO_OF_PED_,NO_OF_PE_1,NO_OF_TOTA,NO_OF_TO_1,LOCATION_O,CRASH_HOUR,CRASH_YEAR,PHYSICAL_J,PHYSICAL_1,RTE_TYPE_C,RTE_TYPE_N,VDOT_DISTR,VDOT_DIS_1,RTE_COMMON
0,1001,101270316,2010-03-03T00:00:00.000Z,2. Angle,Property Damage Crash,1. No Adverse Condition (Clear/Cloudy),2,Not Provided,2,2. Daylight,1. Dry,9. Within Intersection,3. No,0,0,0,0,1. On Roadway,14:30,2010,044,Henry County,SC,Secondary Route,02,Salem,SC-657E (Henry County)
1,1002,101020670,2010-01-30T00:00:00.000Z,4. Sideswipe - Same Direction,Property Damage Crash,6. Snow,2,Not Provided,2,2. Daylight,3. Snowy,1. Main-Line Roadway,3. No,0,0,0,0,1. On Roadway,11:05,2010,029,Fairfax County,IS,Interstate,09,Northern Virginia,I-66E
2,1003,101130763,2010-02-14T00:00:00.000Z,8. Non-Collision,Property Damage Crash,1. No Adverse Condition (Clear/Cloudy),1,Not Provided,1,2. Daylight,1. Dry,8. Non-Intersection,3. No,0,0,0,0,3. Median,14:30,2010,030,Fauquier County,IS,Interstate,07,Culpeper,I-66W
3,1004,101130990,2010-02-15T00:00:00.000Z,9. Fixed Object - Off Road,Property Damage Crash,1. No Adverse Condition (Clear/Cloudy),1,Not Provided,1,1. Dawn,1. Dry,3. Gore Area (b/w Ramp and Highway Edgelines),3. No,0,0,0,0,2. Shoulder,22:17,2010,029,Fairfax County,IS,Interstate,09,Northern Virginia,I-66E Ramp 64A
4,1005,110060009,2010-11-02T00:00:00.000Z,1. Rear End,Property Damage Crash,1. No Adverse Condition (Clear/Cloudy),3,Not Provided,3,2. Daylight,1. Dry,8. Non-Intersection,3. No,0,0,0,0,1. On Roadway,14:27,2010,138,City of Winchester,US,U.S. Route,08,Staunton,US-11S


In [12]:
district_court_speeding_cases_2017.loc[:1000, 'Charge']

0                           84/55 RD
1                           46/25 SP
2                           73/55 SP
3                           56/45 SP
4                           60/35 RD
5                           75/55 SP
6                           77/55 SP
7                           63/40 SP
8                           80/55 SP
9                           84/55 RD
10                          82/55 RD
11                          71/55 SP
12                          74/55 SP
13                          74/50 RD
14                          73/55 SP
15                          74/55 SP
16                          75/45 SP
17                          77/55 SP
18                          73/55 SP
19                          83/55 RD
20                          80/55 RD
21                          81/55 RD
22                 57/35 RD SPEEDING
23                          75/45 RD
24                          62/45 SP
25                          80/55 SP
26                          46/25 SP
2

In [13]:
district_court_speeding_cases_2017[district_court_speeding_cases_2017['person_id'] == 204060000000381]

,index,HearingDate,HearingResult,HearingPlea,HearingContinuanceCode,HearingType,HearingCourtroom,fips,FiledDate,Locality,Status,DefenseAttorney,Address,Gender,Race,Charge,CodeSection,CaseType,Class,OffenseDate,ArrestDate,Complainant,AmendedCharge,AmendedCode,AmendedCaseType,FinalDisposition,SentenceTime,SentenceSuspendedTime,ProbationType,ProbationTime,ProbationStarts,OperatorLicenseSuspensionTime,RestrictionEffectiveDate,RestrictionEndDate,OperatorLicenseRestrictionCodes,Fine,Costs,FineCostsDue,FineCostsPaid,FineCostsPaidDate,VASAP,FineCostsPastDue,person_id,Speed


In [14]:
district_court_speeding_cases_2017['Alleged_Speed'], district_court_speeding_cases_2017['Posted_Speed'] = district_court_speeding_cases_2017['Speed'].str.split('/', 1).str

In [15]:
district_court_speeding_cases_2017['Amended_Speed'] = district_court_speeding_cases_2017['AmendedCharge'].str.extract(r'([0-1]?\d\d/\d*)')
district_court_speeding_cases_2017['Amended_Alleged_Speed'], district_court_speeding_cases_2017['Amended_Posted_Speed'] = district_court_speeding_cases_2017['Amended_Speed'].str.split('/', 1).str

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


## Subset for Hanover and Henrico County only.  FIPS 85 and 87

In [16]:
hanover_henrico_speeding_cases = district_court_speeding_cases_2017[district_court_speeding_cases_2017['fips'].isin([85,87])].reset_index(drop = True)

In [17]:
complainants = hanover_henrico_speeding_cases.groupby('Complainant')['person_id'].nunique().to_frame().reset_index(drop = False)
amended_charges = hanover_henrico_speeding_cases.groupby('AmendedCharge')['person_id'].nunique()
status = hanover_henrico_speeding_cases.groupby('Status')['person_id'].nunique()
charges = hanover_henrico_speeding_cases.groupby('Charge')['person_id'].nunique()
case_types = hanover_henrico_speeding_cases.groupby('CaseType')['person_id'].nunique()
code_sections = hanover_henrico_speeding_cases.groupby('CodeSection')['person_id'].nunique()

### Complainants is not normalized.  Use Fuzzy matching algorithm to create person ID for the complainants

In [18]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [19]:
fuzzy_complainants = complainants['Complainant'].tolist()

In [20]:
fuzzy_complainants

['ADAMS, R.L',
 'AGEE, R T',
 'AGUILLON, C.',
 'ALDERSON, J',
 'ALDERSON, J.E.',
 'ALLEN, D',
 'ALLEN, H',
 'ANCHOR, G',
 'ANCHOR, GM',
 'ANDREWS, C',
 'ANDREWS, C.B.',
 'ANDREWS, R',
 'ANDREWS, R D',
 'ANGELIDIS, D T',
 'ASKEW, J',
 'AYERS, N',
 'AYRES, K L',
 'BAILEY',
 'BAILEY, D A',
 'BAILEY, D.A.',
 'BAILEY, P',
 'BANKS, B M',
 'BANKS, B.',
 'BANKS, BM',
 'BARNES, A G',
 'BARNES, A.G.',
 'BARNES, AG',
 'BARNETT, D',
 'BARRETT, R',
 'BARTON, D',
 'BASIC, C',
 'BASIC, C K',
 'BELCHER, A.M',
 'BELL, R',
 'BENNER, A L',
 'BERNACKI, J. D.',
 'BERNACKI, J.D.',
 'BERRY, M.R',
 'BIVINS, AM',
 'BLUE, J',
 'BOKINGER, C.R.',
 'BOLINGER, C. R.',
 'BOLINGER, C.R.',
 'BOLLAND, J',
 'BOLLAND, J D',
 'BONDS, N B',
 'BONIFACIO, J S',
 'BONISTALLI, G',
 'BOWDEN, K',
 'BOYKIN, N',
 'BRACKETT, J I',
 'BRANDMAIER, J. C.',
 'BREEDEN, R',
 'BRENNAN, M',
 'BRIGHAM, N.',
 'BRILL, S',
 'BROCKENBROUGH, J C',
 'BROOKS, RK',
 'BRUMMITT, J',
 'BRUMMITT, J.S.',
 'BRUSH, J.D.',
 'BURKETT, S',
 'BURRUSS, R',
 'BU

In [21]:
#This function will go row by row to compare names of complainants and assign an ID.  Close matches are
# given the same ID

def create_person_id(names):
    comp_id = 00000
    comp_ids = [comp_id]
    i = 0
    while i < len(names)-1:
        score = fuzz.partial_ratio(names[i], names[i+1])
        if score > 85:
            comp_ids.append(comp_id)
            i +=1
        else:
            comp_ids.append(comp_id + 1)
            i +=1
            comp_id = comp_id + 1
    return comp_ids        

In [22]:
# Append a Complainant ID to hanover/Henrico data frame

complainaint_ids = pd.DataFrame({'Complainant': fuzzy_complainants, 'complainant_id': create_person_id(fuzzy_complainants)})

hanover_henrico_speeding_cases = hanover_henrico_speeding_cases.merge(complainaint_ids, on= 'Complainant', how ='left').reset_index(drop =True)

In [23]:
complainaint_ids

,Complainant,complainant_id
0,"ADAMS, R.L",0
1,"AGEE, R T",1
2,"AGUILLON, C.",2
3,"ALDERSON, J",3
4,"ALDERSON, J.E.",3
5,"ALLEN, D",4
6,"ALLEN, H",4
7,"ANCHOR, G",5
8,"ANCHOR, GM",5
9,"ANDREWS, C",6


# Bokeh Visualizations

In [75]:
complainants = pd.DataFrame(complainants).reset_index()

In [76]:
complainants

,Complainant,person_id
0,"ADAMS, R.L",5
1,"AGEE, R T",19
2,"AGUILLON, C.",7
3,"ALDERSON, J",137
4,"ALDERSON, J.E.",1
5,"ALLEN, D",15
6,"ALLEN, H",34
7,"ANCHOR, G",1
8,"ANCHOR, GM",52
9,"ANDREWS, C",1


In [102]:
complainants_min_fiftytwo = complainants[complainants['person_id'] >= 52].sort_values(by = 'person_id', ascending = False).reset_index(drop = True)
top_ten_complainants = complainants_min_fiftytwo.loc[:9,].reset_index(drop = True)

In [103]:
bokeh_complainants = top_ten_complainants['Complainant'].tolist()
bokeh_count = top_ten_complainants['person_id'].tolist()

In [104]:
# Set the x_range to the list of categories above
p = figure(x_range=bokeh_complainants, plot_height=250, title="Fruit Counts")

# Categorical values can also be used as coordinates
p.vbar(x=bokeh_complainants, top=bokeh_count, width=0.9)

# Set some properties to make the plot look better
p.xgrid.grid_line_color = None
p.y_range.start = 0

show(p)